In [2]:
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import ParameterGrid
from torch.utils.data import Dataset, DataLoader

import datasets

In [4]:
import import_ipynb
from CNN import CNN_3_class

In [12]:
class Net_wrapper:
    """ 
    Wrapper for neural network model. It combines the model itself (nn.Module) together with
    optimizer, loss function and training parameters (such as max_epochs, learning rate and batch size)
    """
        
    def __init__(self, model=CNN_3_class, criterion=nn.CrossEntropyLoss, optimizer=optim.Adam,
                 max_epochs=2, batch_size=32, learning_rate=0.001, **kwargs):
        self.model_params = kwargs
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.max_epochs = max_epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
    
    def __setattr__(self, name, value):
        self.__dict__[name] = value

    def score(self, train_dataset, val_dataset):
        """
        Train model on train_dataset and calculate validation acurracy on val_dataset. 
        """
        if self.model_params:
            model = self.model(self.model_params)
            
        else:
            model = self.model()
            
        optimizer = self.optimizer(model.parameters(), lr=self.learning_rate)
        
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=self.batch_size, shuffle=False)

        for epoch in range(self.max_epochs):
            # print(f'Epoch {epoch+1}/{self.max_epochs}')
            train_accuracies = []
            for data in train_loader:
                results = model.train_step(data, optimizer, self.criterion())
                train_accuracies.append(results['accuracy'].item())

            # Calculate average training loss and accuracy for the epoch
            avg_train_accuracy = sum(train_accuracies) / len(train_accuracies)
            # print(f'Train loss: {avg_train_loss:.4f}, Train accuracy: {avg_train_accuracy:.4f}')

            # Test the model
            val_accuracies = []
            
            with torch.no_grad():
                for data in val_loader:
                    results = model.test_step(data, self.criterion())
                    val_accuracies.append(results['accuracy'].item())

            # Calculate average test loss and accuracy for the epoch
            avg_validation_accuracy = sum(val_accuracies) / len(val_accuracies)
            # print(f'Validation loss: {avg_validation_loss:.4f}, Validation accuracy: {avg_validation_accuracy:.4f}')

        return avg_train_accuracy, avg_validation_accuracy
        
    

In [13]:
class GridSearch():
    """
    Class used to perform grid search on neural networks 

    Attributes:
    self.net - Net_wrapper instance
    self.param_grid - dictionary of parameters we want to search
    self.scores - list for scores of each set of parameters
    self.best_score - best score out of all parameters
    self.best_params - best set of parameters
    self.verbose - if set to 1 additional information (parameter set and accuracy) prints with each iteration of grid search. 
    """
    def __init__(self, net: Net_wrapper, param_grid, verbose=1):
        """

        """
        self.net = net
        self.param_grid = ParameterGrid(param_grid)
        self.scores = []
        self.best_score = 0
        self.best_params = None
        self.verbose = verbose

    def fit(self, train_dataset, val_dataset):
        """
        Fit the grid search with train and validation dataset. 
        Search for optimal parameters for neural network declared during 
        initialization of GridSearch instance.
        """
        
        for params in self.param_grid:
            for hyp_name, hyp_val in params.items():
                if hasattr(self.net, hyp_name):
                    setattr(self.net, hyp_name, hyp_val)
                else:
                    setattr(self.net.model, hyp_name, hyp_val)
            _, val_accuracy = self.net.score(train_dataset, val_dataset)
            self.scores.append(val_accuracy)
            if val_accuracy > self.best_score:
                self.best_score = val_accuracy
                self.best_params = params
            if self.verbose == 1:
                print('Parameter set:', params)
                print(f'val_accuracy: {val_accuracy:.4f}')
        
        return self

### Example hyperparameters (from most to least important):

In [5]:
hyperparams1 = {'learning_rate': [0.001, 0.005, 0.01],
               'batch_size': [8, 32, 64],
               'max_epochs': [5, 10, 15]}

hyperparams2 = {
                'no_neurons': [25, 50, 100],
                'kernel_size': [2, 3, 5],
                'number_of_filters': [16, 32, 64]
                }

hyperparams3 = {
               'optimizer': [optim.Adam, optim.Adagrad, optim.SGD],
               'activation_func': [nn.ReLU, nn.LeakyReLU, nn.Tanh],
               'dropout_rate': [0.0, 0.25, 0.5]}



### Sample for testing GridSearch

In [8]:
train_dataset = datasets.cifar_train
val_dataset = datasets.cifar_val

In [9]:
from torch.utils.data import DataLoader, SubsetRandomSampler, Subset

subset_indices = list(range(500))
subset_sampler = SubsetRandomSampler(subset_indices)

subset_train_dataset = Subset(train_dataset, subset_indices)
subset_val_dataset = Subset(val_dataset, subset_indices)

### GridSearch test

In [14]:
test_hyper_params = {'lr': [0.001, 0.005], 'batch_size': [8, 16], 'max_epochs': [3, 5]}
my_net = Net_wrapper()
gs = GridSearch(net=my_net, param_grid=test_hyper_params, verbose=1)
gs = gs.fit(subset_train_dataset, subset_val_dataset)

Parameter set: {'batch_size': 8, 'lr': 0.001, 'max_epochs': 3}
val_accuracy: 0.2302
Parameter set: {'batch_size': 8, 'lr': 0.001, 'max_epochs': 5}
val_accuracy: 0.2381
Parameter set: {'batch_size': 8, 'lr': 0.005, 'max_epochs': 3}
val_accuracy: 0.2321
Parameter set: {'batch_size': 8, 'lr': 0.005, 'max_epochs': 5}
val_accuracy: 0.3175
Parameter set: {'batch_size': 16, 'lr': 0.001, 'max_epochs': 3}
val_accuracy: 0.2422
Parameter set: {'batch_size': 16, 'lr': 0.001, 'max_epochs': 5}
val_accuracy: 0.2559
Parameter set: {'batch_size': 16, 'lr': 0.005, 'max_epochs': 3}
val_accuracy: 0.2441
Parameter set: {'batch_size': 16, 'lr': 0.005, 'max_epochs': 5}
val_accuracy: 0.2324


In [11]:
print(gs.best_score)
print(gs.best_params)

0.294921875
{'batch_size': 16, 'lr': 0.001, 'max_epochs': 5}
